In [74]:
# export
import csv
import numpy as np  # http://www.numpy.org
import ast
from datetime import datetime
from math import log, floor, ceil
import random
import numpy as np

Modify the Utility class's methods. You can also add additional methods as required but don't change existing methods' arguments.

In [81]:
# export
class Utility(object):
    
    # This method computes entropy for information gain
    def entropy(self, class_y):
        entropy = 0
        frequency = np.bincount(class_y)/len(class_y)
        
        for x in frequency:
            if x>0:
                entropy -= x*np.log2(x)
        
        return entropy


    def partition_classes(self, X, y, split_attribute, split_val):
        X_left = []
        X_right = []

        y_left = []
        y_right = []
        
        for i in range(len(X)):
            if X[i][split_attribute] <= split_val:
                X_left.append(X[i])
                y_left.append(y[i])
            
            else: 
                X_right.append(X[i])
                y_right.append(y[i])
        
        
        return (X_left, X_right, y_left, y_right)


    def information_gain(self, previous_y, current_y):
        info_gain = 0
        info_gain = self.entropy(previous_y)
        
        for split in range(len(current_y)):
            info_gain -= (len(current_y[split])/len(previous_y)) * self.entropy(current_y[split])
        
        return info_gain


    def best_split(self, X, y): 
        X_left, X_right, y_left, y_right = [], [], [], []

        info_best_split = {}
        current_best = 0

        for col in range(len(X[0])): # This iterator accounts for the number of columns available in X
            split_attribute = col  
            column_values = sorted([x[col] for x in X])
            
            for val in column_values:    
                split_val = val
                X_left, X_right, y_left, y_right = self.partition_classes(X, y, split_attribute, split_val)
                current_y = [y_left, y_right]
                info_gain = self.information_gain(y, current_y)
                
                if info_gain > current_best: 
                    current_best = info_gain
                    info_best_split = {"best_split_feature": split_attribute,
                                       "best_split_val": split_val,
                                       "X_left": X_left, 
                                       "X_right": X_right, 
                                       "y_left": y_left, 
                                       "y_right": y_right} 

         
        return info_best_split

            

### Define the classes 'DecisionTree' and 'RandomForest'

Please modify the 'DecisionTree' and 'RandomForest' classes below

In [82]:
# export
class DecisionTree(object):
    def __init__(self, max_depth):
        # Initializing the tree as an empty dictionary or list, as preferred
        self.tree = {}
        self.max_depth = max_depth
        self.util = Utility()
        
    def learn(self, X, y, par_node = {}, depth=0):
        if len(X) == 0 or len(y) == 0:
            self.tree["node_type"] = "fail_leaf"
            self.tree["node_value"] = 0
            return
        
        elif self.util.entropy(y)==0:
            self.tree["node_type"] = "output_leaf"
            self.tree["node_value"] = y[0]
            return
        
        elif (depth == self.max_depth) or len(set(map(tuple, X))) == 1:
            y = np.asarray(y)
            self.tree["node_type"] = "output_leaf"
            self.tree["node_value"] = np.bincount(y).argmax()
            return
        
        best_split = self.util.best_split(X, y)
               
        self.tree['split_attr'] = best_split['best_split_feature'] 
        self.tree['split_val'] = best_split["best_split_val"]
        
        self.tree["left_branch"] = DecisionTree(max_depth=self.max_depth)   
        self.tree["left_branch"].learn(best_split["X_left"],best_split["y_left"], {}, depth+1)
        
        self.tree["right_branch"] = DecisionTree(max_depth=self.max_depth)
        self.tree["right_branch"].learn(best_split["X_right"], best_split["y_right"], {}, depth+1)
        
        if len(self.tree["left_branch"].tree) or len(self.tree["right_branch"].tree) == 0:
            self.tree["node_type"] = "fail_leaf"
            self.tree["node_value"] = 0
            return


    def classify(self, record):
        if self.tree["node_type"] == "output_leaf":
            return self.tree["node_value"]
        
        attr = self.tree["split_attr"]
        val = self.tree["split_val"]
        
        if record[attr] <= val: 
            return self.tree['left_branch'].classify(record)
        else: 
            return self.tree['right_branch'].classify(record)

In [83]:
# export
class RandomForest(object):
    num_trees = 0
    decision_trees = []


    bootstraps_datasets = []


    bootstraps_labels = []

    def __init__(self, num_trees):
        # Initialization done here
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree(max_depth=10) for i in range(num_trees)]
        self.bootstraps_datasets = []
        self.bootstraps_labels = []
        
    def _bootstrapping(self, XX, n):
        # Reference: https://en.wikipedia.org/wiki/Bootstrapping_(statistics)
        #
        # TODO: Create a sample dataset of size n by sampling with replacement
        #       from the original dataset XX.
        # Note that you would also need to record the corresponding class labels
        # for the sampled records for training purposes.

        sample = [] 
        labels = []  
        
        random_ind = random.sample(range(len(XX)), n)
        
        for i in random_ind: 
            sample.append(XX[i][:-1]) 
            labels.append(XX[i][-1]) 

        return (sample, labels)

    def bootstrapping(self, XX):
        # Initializing the bootstap datasets for each tree
        for i in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)

    def fitting(self):
         for num in range(self.num_trees):
            self.decision_trees[num].learn(self.bootstraps_datasets[num], self.bootstraps_labels[num])

    def voting(self, X):
        y = []

        for record in X:
            # Following steps have been performed here:
            #   1. Find the set of trees that consider the record as an
            #      out-of-bag sample.
            #   2. Predict the label using each of the above found trees.
            #   3. Use majority vote to find the final label for this recod.
            votes = []
            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]
                
                if record not in dataset:
                    OOB_tree = self.decision_trees[i]
                    effective_vote = OOB_tree.classify(record)
                    votes.append(effective_vote)

            counts = np.bincount(votes)
        
            if len(counts) == 0:
                votes = [dt.classify(record) for dt in self.decision_trees]
                y.append(np.bincount(votes).argmax())
                
            else:
                y = np.append(y, np.argmax(counts))
                
        return y

    def user(self):
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """

        return 'egrant37'

In [84]:
# export
def get_forest_size():
    forest_size = 10
    return forest_size

def get_random_seed():
    random_seed = 0
    return random_seed

### Do not modify the below cell
The cell below is provided to test that your random forest classifier can be successfully built and run. Similar steps will be used to build and run your code in Gradescope. Any additional testing of functions can be done in the cells below the `%run helpers/notebook2script submission` cell, as these will not be parsed by the autograder.

In [85]:
def run():
    np.random.seed(get_random_seed())
    # start time 
    start = datetime.now()
    X = list()
    y = list()
    XX = list()  # Contains data features and data labels
    numerical_cols = set([i for i in range(0, 31)])  # indices of numeric attributes (columns)

    # Loading data set
    print("reading the data")
    with open("Wisconsin_breast_prognostic.csv") as f:
        next(f, None)
        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                if i in numerical_cols:
                    xline.append(ast.literal_eval(line[i]))
                else:
                    xline.append(line[i])

            X.append(xline[:-1])
            y.append(xline[-1])
            XX.append(xline[:])

    # Initializing a random forest.
    randomForest = RandomForest(get_forest_size())

    # printing the name
    print("__Name: " + randomForest.user()+"__")

    # Creating the bootstrapping datasets
    print("creating the bootstrap datasets")
    randomForest.bootstrapping(XX)

    # Building trees in the forest
    print("fitting the forest")
    randomForest.fitting()

    # Calculating an unbiased error estimation of the random forest
    # based on out-of-bag (OOB) error estimate.
    y_predicted = randomForest.voting(X)

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print("accuracy: %.4f" % accuracy)
    print("OOB estimate: %.4f" % (1 - accuracy))

    # end time
    print("Execution time: " + str(datetime.now() - start))

In [86]:
# Call the run() function to test your implementation
# Use this cell and any cells below for additional testing
run()

reading the data
__Name: egrant37__
creating the bootstrap datasets
fitting the forest
accuracy: 1.0000
OOB estimate: 0.0000
Execution time: 0:01:05.666269
